<div align="right">Python [conda env:PY27_Test]</div>

What if you want to use the name of things in your code to do something?  The most common use case would be to pass a value into a function and then have that function ouput the name of the variable containing the original value as well as the desired output.  Additionally this could allow us to track function calls to other functions by outputting the names of what function got called when it gets used.

Much of the content in this notebook was proposed or inspired on this posting thread: [Stack Overflow Post On this Topic](http://stackoverflow.com/a/21339843/7525365)

## TOC

- [Tests with dill library - getname()](#getname)
- [Using globals() and .items() to get Names](#globals)
- [Simplest Answer to get Names of Methods](#__name__)
- [Looping over variables in globals() to get Values](#loop_4_values)


<a id="getname" name="getname"></a>
### Tests with dill library - getname()

In [1]:
from dill.source import getname    # run this cell first before any cells below

peg1 = [1,2]
peg2 = [3]
peg3 = [5,4]

In [2]:
# this example shows what at first would appear to be unexpected behavior
# it illustrates some concepts though later cells of this notebook show how to effectively use dill.getname

def move_from(source, target):
    # this did not work at all:
    print("Does not work:\n Move %d from %s to %s next.\n" %(source[-1], getname(source), getname(target)))
    print("Resolves to stored Value, not the names:")
    print("Move %d from %s to %s next." %(source[-1], getname(*source), getname(*target)))
    
move_from(peg1, peg3)   

Does not work:
 Move 2 from None to None next.

Resolves to stored Value, not the names:
Move 2 from 1 to 5 next.


In [3]:
print(getname(peg1))  # note how no value is returned for variables

None


In [4]:
# but the name is returned for functions
getname(move_from)

'move_from'

In [5]:
def test_another_way_fun(fun, x, y):
    fun(x, y)
    return getname(fun)

test_another_way_fun(move_from, peg1, peg2)  # only move_from() resolves to its original name

Does not work:
 Move 2 from None to None next.

Resolves to stored Value, not the names:
Move 2 from 1 to None next.


'move_from'

<a id="globals" name="globals"></a>
### Using globals() and .items() to get Names

In [6]:
### try these lines after some variables and functions are defined:
# globals()
# globals().items()
### result is long and rather messy ... 

# this code is from the stack overflow post: http://stackoverflow.com/a/1538399/7525365

def variable_for_value(value):
    for n,v in globals().items():
        if v == value:
            return n
    return None

variable_for_value(peg1)

'peg1'

In [7]:
variable_for_value(move_from)

'move_from'

In [8]:
print(variable_for_value(peg1[0]))  # cannot pass in an index or pointer and get the name
print(variable_for_value(*peg2))    # None is returned

None
None


In [9]:
# this is probably because it is treating the above literally as if you were asking for
# thing named '*peg2' or thing named peg1[0]
# thing doesn't exist?  It does this:

x = variable_for_value('does_not_exist') # to even get nonexistent thing to pass in have to wrap in quotes and treat as string
                                         # otherwise, NameError is triggered before we can see how it handles it
print(x)

None


In [10]:
def move_from(source, target):
    print("Move %d from %s to %s next." %(source[-1], getname(*source), getname(*target)))

def test_another_way_fun(fun, x, y):
    fun(x, y)
    lst = [getname(fun), variable_for_value(x), variable_for_value(y)]
    return lst

test_another_way_fun(move_from, peg1, peg3)

Move 2 from 1 to 5 next.


['move_from', 'peg1', 'peg3']

In [11]:
# using just variable_for_value

def move_from(source, target):
    print("Move %d from %s to %s next." %(source[-1], getname(*source), getname(*target)))

def test_another_way_fun(fun, x, y):
    fun(x, y)
    lst = [variable_for_value(fun), variable_for_value(x), variable_for_value(y)]
    return lst

test_another_way_fun(move_from, peg1, peg3)

Move 2 from 1 to 5 next.


['move_from', 'peg1', 'peg3']

In [12]:
'''Odd symptom to be aware of ... in earlier tests, somehow when re-running one of the cells above,
   the last value returned as "_" instead of its true name.  Despite numerous edits to code in this cell
   to attempt to replicate the symtpom, it does not seem to recur.  It is not known if this is a random glitch
   or something that may occur again under the right coditions. '''

def test_another_way_fun_multiTest(fun, x, y):
    fun(x, y)
    lst =  [getname(fun), variable_for_value(fun), variable_for_value(x), variable_for_value(y)]
    lst2 = [getname(fun), variable_for_value(fun), variable_for_value(x), variable_for_value(y)]
    lst3 = [lst, lst2]
    return lst3

test_another_way_fun_multiTest(move_from, peg1, peg3)

Move 2 from 1 to 5 next.


[['move_from', 'move_from', 'peg1', 'peg3'],
 ['move_from', 'move_from', 'peg1', 'peg3']]

<a id="__name__" name="__name__"></a>
### __name__ of Methods

In [13]:
# shortcut when you know that it is a method being passed in:

def pass_in_someMethod(fun):
    return fun.__name__

pass_in_someMethod(test_another_way_fun_multiTest)

'test_another_way_fun_multiTest'

In [14]:
# can't do this with variables (.__name__ attribute is only on methods for built-ins of the language) ...
try:
    pass_in_someMethod(peg1)
except Exception as ee:
    print(type(ee))
    print(ee)

<type 'exceptions.AttributeError'>
'list' object has no attribute '__name__'


In [63]:
# in action ... finding out what got passed in to functions:

def variable_for_value2(value):
    # a copy of value_for_variable to test something
    for n,v in globals().items():
        if v == value:
            return n
    return None

def test_functions(fun, *args):
    # this function assumes that *args is either:
    #   * a single function or variable being passed in
    #   * 2 variables that form the parameters
    
    print("Function call:\n" + ("-"*32))
    # print(len(args))
    
    if len(args) == 1:
        if str(type(*args)) == "<type 'function'>":
            str_args = str(*args)
        else:
            str_args = variable_for_value(*args)
    else:
        str_args = variable_for_value(args[0]) + ", " + variable_for_value(args[1])
    
    print(fun.__name__ + "(" + str_args + ")" + "\n" + ("-"*32))
    print("Return Value: ")
    print(fun(*args))
    
# For use in testing some functions:

def justSayit(addTxt):
    rtnVal= "Say it " + addTxt + "."
    return rtnVal

txt = "with love"

funList = [pass_in_someMethod, move_from, variable_for_value, justSayit]
          #  1 arg             2 args       1 arg             1 args 

funList2 = [pass_in_someMethod, variable_for_value2, variable_for_value ]    

In [64]:
# funList2: is list of functions that can all return the name of a function passed into them
#           each is given the same function to evaluate
#           one unexpected symptom:  when the function gets its own name, if returns the fn argument that did it
#           all other tests, the original function name passed in to thest function is returned

print("Ouput below is function call first, then return of function (both functions return names in this case):")
for fn in funList2:
    # in real world: loop like this would use same type and number of input for each fun
    test_functions(fn, variable_for_value)
    print("")

Ouput below is function call first, then return of function (both functions return names in this case):
Function call:
--------------------------------
pass_in_someMethod(<function variable_for_value at 0x00000000042026D8>)
--------------------------------
Return Value: 
variable_for_value

Function call:
--------------------------------
variable_for_value2(<function variable_for_value at 0x00000000042026D8>)
--------------------------------
Return Value: 
variable_for_value

Function call:
--------------------------------
variable_for_value(<function variable_for_value at 0x00000000042026D8>)
--------------------------------
Return Value: 
variable_for_value



In [66]:
# more tests using funList which has some different things to test with

print("Ouput below is function call first, then return of function:")
print("#"*32)

test_functions(funList[0], test_another_way_fun_multiTest)
print("")

test_functions(funList[1], peg1, peg3)
print("")

test_functions(funList[2], peg2)
print("")

test_functions(funList[3], txt)
print("")

Ouput below is function call first, then return of function:
################################
Function call:
--------------------------------
pass_in_someMethod(<function test_another_way_fun_multiTest at 0x0000000003DAEF98>)
--------------------------------
Return Value: 
test_another_way_fun_multiTest

Function call:
--------------------------------
move_from(peg1, peg3)
--------------------------------
Return Value: 
Move 2 from 1 to 5 next.
None

Function call:
--------------------------------
variable_for_value(peg2)
--------------------------------
Return Value: 
peg2

Function call:
--------------------------------
justSayit(txt)
--------------------------------
Return Value: 
Say it with love.



In [69]:
# turning some of this code into an object

class GetDeclaredName(object):
    '''Get Name of Variable or Function Passed Into Object. Use inside methods to identify what got passed in via args.'''
    def name_of_declaredEement(self, value):
        ''' name_of_declaredElement -->\n\nreturns name of declared element (variable or function) passed into it.'''
        # code modified from this Stack Overflow post:  http://stackoverflow.com/a/1538399/7525365
        for n,v in globals().items():
            if v == value:
                return n
        return None
    
    ## idea for future development:  add in test_function that can take any combination of arguments

In [73]:
gdn = GetDeclaredName()
GetDeclaredName.name_of_declaredEement(gdn, variable_for_value)  # Python 2.7 and 3.6 compliant syntax

'variable_for_value'

In [74]:
animal = "dog"
GetDeclaredName.name_of_declaredEement(gdn, animal)  # Python 2.7 and 3.6 compliant syntax

'animal'

<a id="loop_4_values" name="loop_4_values"></a>
### Looping over variables in globals() to get Values

In [67]:
# getting the value of variables by their name on globals()

x = peg1
y = peg2
z = peg3

my_list = ["x", "y", "z"] # x, y, z have been previously defined

for name in my_list:
    print("handling variable %s" %name)
    bla = globals()[name]  # accessing the value at the index for name
    print("bla: %s" %bla)

handling variable x
bla: [1, 2]
handling variable y
bla: [3]
handling variable z
bla: [5, 4]
